In [ ]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm


In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.durunubi.kr/4-1-1-walk-Road-view.do?pageNum=&theme_mng=T_ROUTE_MNG0000000047')

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [ ]:
# h4 이름, h6 지역
result = soup.select(".col-xl-3")


In [ ]:
result[0].select_one("a").attrs["onclick"][10:29]

In [ ]:
def crawler_all(page:int):
    # print(f"페이지 {page} 시작")  # 진행 확인용
    streets = []


    res = requests.get(f'https://www.durunubi.kr/road-walk.do?curPage={page}&keyword=')
    soup = BeautifulSoup(res.text, 'html.parser')
    result = soup.select(".col-xl-3")
    
    for item in result:
        street = {}
        street["산책길"] = item.find('h4').text
        code = item.select_one("a").attrs["onclick"][21:42]
        street["산책길 코드"] = code
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(f'https://www.durunubi.kr/4-1-1-walk-Road-view.do?pageNum=&theme_mng={code}')
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        courses = soup.select(".col-xl-3")
        
        for i, course in enumerate(courses):
            street[f"코스{i+1}"] = course.find('h4').text
            street[f"지역{i+1}"] = course.find('h6').text
            street[f"코스코드{i+1}"] = course.select_one("a").attrs["onclick"][10:29]

        driver.quit()
        streets.append(street)
        
    # print(f"페이지 {page} 끝")  # 진행 확인용
    return streets

In [ ]:
crawler_all(1)

In [ ]:



def crawler_all(start_page: int, end_page: int):
    """
    start_page부터 end_page까지 크롤링한 결과를 리스트로 반환
    """    
    streets = []

    # ✅ 크롬 드라이버를 한 번만 실행
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    for page in tqdm(range(start_page, end_page + 1), desc=f"Pages {start_page}~{end_page}"):
        # 1) 리스트 페이지 가져오기
        res = requests.get(f'https://www.durunubi.kr/road-walk.do?curPage={page}&keyword=')
        soup = BeautifulSoup(res.text, 'html.parser')
        result = soup.select(".col-xl-3")

        for item in result:
            street = {}
            street["산책길"] = item.find('h4').text
            code = item.select_one("a").attrs["onclick"][21:42]
            street["산책길 코드"] = code

            # 2) 디테일 페이지 크롬 드라이버로 열기
            driver.get(f'https://www.durunubi.kr/4-1-1-walk-Road-view.do?pageNum=&theme_mng={code}')
            time.sleep(3)

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            courses = soup.select(".col-xl-3")

            for i, course in enumerate(courses):
                street[f"지역{i+1}"] = course.find('h6').text
                street[f"코스{i+1}"] = course.find('h4').text
                street[f"코스코드{i+1}"] = course.select_one("a").attrs["onclick"][10:29]

            streets.append(street)

    # ✅ 마지막에 한 번만 닫기
    driver.quit()

    return streets


In [ ]:
crawler_all(1)

In [ ]:
import pandas as pd

if __name__ == "__main__":
    start_time = time.time()

    # ✅ 페이지를 나눠서 실행 (예: 1~15, 16~30, 31~45)
    results = []
    for s, e in [(1, 15), (16, 30), (31, 45)]:
        data = crawler_all(s, e)
        results.extend(data)

    # ✅ DataFrame으로 변환
    df = pd.DataFrame(results)

    # # ✅ CSV 저장
    # df.to_csv("durunubi_results.csv", index=False, encoding="utf-8-sig")

    print(f"총 {len(df)}개 데이터 크롤링 완료!")
    print("소요시간:", time.time() - start_time, "초")

In [ ]:
file_path = '두루누비_서울확인용.json'

In [ ]:
import json

In [ ]:
with open(file_path, 'w') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [ ]:
# ✅ CSV 저장
df.to_csv("durunubi_results.csv", index=False, encoding="utf-8-sig")


In [ ]:
len(results)

In [2]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from tqdm import tqdm

In [1]:
import pymysql
from dotenv import load_dotenv
import os 

load_dotenv()

# 팀원 계정 정보
conn = pymysql.connect(
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT")),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    db=os.getenv("DB_NAME"),
    charset=os.getenv("DB_CHARSET", "utf8mb4")
)
cur = conn.cursor(pymysql.cursors.DictCursor)
cur.execute("""select c.ID, t.ThemeName, c.Region, c.Course, c.CourseCode
            from tb_seoul_courses as c
            join tb_seoul_trails as t
            on c.TrailsID = t.ID
            where Region LIKE '서울%';""")
seoul_courses = cur.fetchall()

cur.close()
conn.close()

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))        

code = seoul_courses[0]['CourseCode']
# 2) 디테일 페이지 크롬 드라이버로 열기
driver.get(f'https://www.durunubi.kr/4-2-1-1-walk-mobility-view-detail.do?crs_idx={code}')
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
content0 = soup.select('#appVue > section.page-title > div > ul > li', class_="list-inline-item")
content1 = soup.select('.pb-50')
info = [info.rstrip().lstrip() for rinfo in content0 for info in rinfo.text.split('\n') if info.rstrip().lstrip() != '']
print(info)
new_info = ' '.join(info).split(' ')
print(new_info)



['길이 18.6km', '소요시간', '8시간', '10분', '난이도', '어려움', '비순환형']
['길이', '18.6km', '소요시간', '8시간', '10분', '난이도', '어려움', '비순환형']


In [ ]:
for rinfo in content0:
    for info in rinfo.text.split('\n'):
        if info.rstrip().lstrip() != '':
            info.rstrip().lstrip()


['길이 18.6km']
['소요시간', '                    ', '                    ', '                        8시간', '                    ', '                    ', '                        10분', '                    ']
['난이도', '                    ', '                    ', '                    어려움']
['비순환형']


['길이 18.6km', '소요시간', '8시간', '10분', '난이도', '어려움', '비순환형']

In [ ]:
# content1[0]
# soup.select('#appVue > section.page-title > div > ul > li', class_="list-inline-item")
content = [sentence.rstrip().lstrip() for sentence in content1[0].text.split('\n')]
util = [sentence.rstrip().lstrip() for sentence in content1[1].text.split('\n')]

In [60]:
content

['코스소개 코스개요',
 '- 수락산과 불암산을 연결하는 코스로 산림욕을 즐길 수 있음',
 '- 채석장부지 전망대에서 바라보는 전망이 매우 뛰어남',
 '- 난이도가 높은 편으로 도보여행 시 단단한 준비가 필요한 코스',
 '관광포인트',
 "- 총면적 52,417㎡에 꽃창포를 비롯한 다양한 꽃을 볼 수 있는 생태공원 '서울창포원'",
 '- 산 속에 이런 곳이? ‘불암산 전망대’',
 "- 푸른 바위와 안개가 자욱한 계곡이란 뜻의 수락곡계곡 '벽운동계곡'",
 "- 신라 진평왕 43년(621년)에 원광국사가 삼국통일의 염원을 담아 창건한 '수암사'",
 "- 1872년 명성황후의 염원에 의해 조성된 높이 22.7m, 폭 7m의 '마애관음보살좌상'(서울시유형문화재 제 124호)",
 '여행정보(주의사항)',
 '- 지하철 1호선, 7호선 도봉산역 2번 출구',
 '- 창포원공원 시작점에 서울둘레길 안내센터(창포원)가 있어 스탬프북, 안내지도 등을 받을 수 있고 완주 후에는 완주증을 받을 수 있음',
 '- 화장실이 많지 않아 지하철에서 미리 다녀올 것을 추천함',
 '- 당고개갈림길에서 하산하여 당고개역을 통과하는 단축코스를 이용할 수 있음',
 '* 도보인증안내 : http://gil.seoul.go.kr/walk/main.jsp',
 '#산  #숲  #가을단풍  #비순환형  #단체  #지하철']

In [65]:
len(content), content.index('관광포인트'), content.index('여행정보(주의사항)')

(17, 4, 10)

In [82]:
# 길 정보
sight_idx = content.index('관광포인트')
trevel_idx = content.index('여행정보(주의사항)')
course_info = content[:sight_idx]
sight_info = content[sight_idx:trevel_idx]
trevel_info = content[trevel_idx:-1]
keywords = [key.rstrip().lstrip() for key in content[-1].split('#') if key != '']

In [85]:
new_util = [i for i in util if i != '']
new_util

['주요지점 및 편의시설',
 '코스 경로',
 '도봉산역 - 서울창포원 - 덕릉고개 - 넓은마당 - 경수사 - 성관사 - 넓적바위 - 학도암 - 태릉 - 화랑대역',
 '식수',
 '사전에 준비',
 '매점',
 '없음']

In [91]:
# 시설물 정보
main_index = 1
water_index = new_util.index('식수')
store_index = new_util.index('매점')
main_point = new_util[main_index:water_index]
water_info = new_util[water_index:store_index]
store_info = new_util[store_index:]

In [103]:
info = [i.rstrip().lstrip() for i in content0[0].text.split('\n') if i.rstrip().lstrip() != '']
info

['길이 18.6km 소요시간', '8시간', '10분', '난이도', '어려움 비순환형']

In [108]:
new_info = ' '.join(info).split(' ')
new_info

['길이', '18.6km', '소요시간', '8시간', '10분', '난이도', '어려움', '비순환형']

In [ ]:
time_idx = new_info.index('소요시간')
level_index = new_info.index('난이도')
length = new_info[:time_idx]
times = new_info[time_idx:level_index]
level = new_info[level_index:-1]
route_type = new_info[-1]

'비순환형'

In [7]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

infos = []

# ✅ 크롬 드라이버를 한 번만 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for course in tqdm(seoul_courses):
    detail = {}
    code = course['CourseCode']
    # 2) 디테일 페이지 크롬 드라이버로 열기
    driver.get(f'https://www.durunubi.kr/4-2-1-1-walk-mobility-view-detail.do?crs_idx={code}')
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    content0 = soup.select('#appVue > section.page-title > div > ul > li', class_="list-inline-item")
    content1 = soup.select('.pb-50')
    
    # 메인 정보
    info = [info.rstrip().lstrip() for rinfo in content0 for info in rinfo.text.split('\n') if info.rstrip().lstrip() != '']
    new_info = ' '.join(info).split(' ')
    time_idx = new_info.index('소요시간')
    level_index = new_info.index('난이도')
    length = new_info[:time_idx]
    times = new_info[time_idx:level_index]
    level = new_info[level_index:-1]
    route_type = new_info[-1]
    
    detail['길이'] = length[1]
    detail['소요시간'] = ' '.join(times[1:])
    detail['난이도'] = level[1]
    detail['길 타입'] = route_type

    # 길 정보
    content = [sentence.rstrip().lstrip() for sentence in content1[0].text.split('\n')]
    sight_idx = content.index('관광포인트')
    trevel_idx = content.index('여행정보(주의사항)')
    course_info = content[:sight_idx]
    sight_info = content[sight_idx:trevel_idx]
    trevel_info = content[trevel_idx:-1]
    keywords = [key.rstrip().lstrip() for key in content[-1].split('#') if key != '']
    
    detail['코스소개'] = ', '.join(course_info[1:])
    detail['관광포인트'] = ', '.join(sight_info[1:])
    detail['여행정보'] = ', '.join(trevel_info[1:])
    detail['키워드'] = ', '.join(keywords)

    # 시설물 정보
    util = [sentence.rstrip().lstrip() for sentence in content1[1].text.split('\n')]
    new_util = [i for i in util if i != '']
    main_index = 1
    if '식수' in new_util:
        water_index = new_util.index('식수')
    else:
        water_index = len(new_util)
    if '매점' in new_util:
        store_index = new_util.index('매점')
    else:
        store_index = len(new_util)
    main_point = new_util[main_index:water_index]
    if new_util[water_index:store_index]:
        water_info = new_util[water_index:store_index]
    else:
        water_info = '사전에 준비'
    if new_util[store_index:]:
        store_info = new_util[store_index:]
    else:
        store_info = '없음'

    detail['코스 경로'] = main_point[1]
    detail['식수'] = water_info[1]
    detail['매점'] = store_info[1]
    
    infos.append(detail)

# ✅ 마지막에 한 번만 닫기
driver.quit()

for i, con in enumerate(infos):
    seoul_courses[i].update(con)

100%|██████████| 139/139 [08:57<00:00,  3.87s/it]


In [140]:
seoul_courses

[{'ID': 1,
  'ThemeName': '서울둘레길',
  'Region': '서울 노원구',
  'Course': '서울둘레길 1코스 수락·불암산코스',
  'CourseCode': 'T_CRS_MNG0000003591'},
 {'ID': 2,
  'ThemeName': '서울둘레길',
  'Region': '서울 노원구',
  'Course': '서울둘레길 2코스 용마·아차산코스',
  'CourseCode': 'T_CRS_MNG0000003592'},
 {'ID': 3,
  'ThemeName': '서울둘레길',
  'Region': '서울 강동구',
  'Course': '서울둘레길 3코스 고덕·일자산코스',
  'CourseCode': 'T_CRS_MNG0000003593'},
 {'ID': 4,
  'ThemeName': '서울둘레길',
  'Region': '서울 강남구',
  'Course': '서울둘레길 4코스 대모·우면산코스',
  'CourseCode': 'T_CRS_MNG0000003594'},
 {'ID': 5,
  'ThemeName': '서울둘레길',
  'Region': '서울 금천구',
  'Course': '5코스 관악·호암산코스',
  'CourseCode': 'T_CRS_MNG0000003595'},
 {'ID': 6,
  'ThemeName': '서울둘레길',
  'Region': '서울 금천구',
  'Course': '6코스 안양천·한강코스',
  'CourseCode': 'T_CRS_MNG0000000899'},
 {'ID': 7,
  'ThemeName': '서울둘레길',
  'Region': '서울 강서구',
  'Course': '7코스 봉산·앵봉산코스',
  'CourseCode': 'T_CRS_MNG0000003596'},
 {'ID': 8,
  'ThemeName': '서울둘레길',
  'Region': '서울 은평구',
  'Course': '8코스 북한·도봉산코스',
  'CourseCode': 

In [6]:
seoul_courses

[{'ID': 1,
  'ThemeName': '서울둘레길',
  'Region': '서울 노원구',
  'Course': '서울둘레길 1코스 수락·불암산코스',
  'CourseCode': 'T_CRS_MNG0000003591',
  '길이': '18.6km',
  '소요시간': '8시간 10분',
  '난이도': '어려움',
  '길 타입': '비순환형',
  '코스소개': '- 수락산과 불암산을 연결하는 코스로 산림욕을 즐길 수 있음,- 채석장부지 전망대에서 바라보는 전망이 매우 뛰어남,- 난이도가 높은 편으로 도보여행 시 단단한 준비가 필요한 코스',
  '관광포인트': "- 총면적 52,417㎡에 꽃창포를 비롯한 다양한 꽃을 볼 수 있는 생태공원 '서울창포원',- 산 속에 이런 곳이? ‘불암산 전망대’,- 푸른 바위와 안개가 자욱한 계곡이란 뜻의 수락곡계곡 '벽운동계곡',- 신라 진평왕 43년(621년)에 원광국사가 삼국통일의 염원을 담아 창건한 '수암사',- 1872년 명성황후의 염원에 의해 조성된 높이 22.7m, 폭 7m의 '마애관음보살좌상'(서울시유형문화재 제 124호)",
  '여행정보': '- 지하철 1호선, 7호선 도봉산역 2번 출구,- 창포원공원 시작점에 서울둘레길 안내센터(창포원)가 있어 스탬프북, 안내지도 등을 받을 수 있고 완주 후에는 완주증을 받을 수 있음,- 화장실이 많지 않아 지하철에서 미리 다녀올 것을 추천함,- 당고개갈림길에서 하산하여 당고개역을 통과하는 단축코스를 이용할 수 있음,* 도보인증안내 : http://gil.seoul.go.kr/walk/main.jsp',
  '키워드': '산,숲,가을단풍,비순환형,단체,지하철',
  '코스 경로': '도봉산역 - 서울창포원 - 덕릉고개 - 넓은마당 - 경수사 - 성관사 - 넓적바위 - 학도암 - 태릉 - 화랑대역',
  '식수': '사전에 준비',
  '매점': '없음'},
 {'ID': 2,
  'ThemeName': '서울둘레길',
  'Region': '서울 노원구',


In [9]:
import json

In [ ]:
# with open('두루누비_서울.json', 'w', encoding="utf-8") as f:
#     json.dump(seoul_courses, f, ensure_ascii=False, indent=4)

In [ ]:
# # ✅ CSV 저장
# import pandas as pd
# df = pd.DataFrame(seoul_courses)
# df
# df.to_csv("두루누비_서울.csv", index=False, encoding="utf-8-sig")